In [2]:
# Imports and data reading from files
import pandas as pd
from collections import Counter

data = pd.read_csv('../../data/games/2021-22.csv')
players = pd.read_csv('../../data/players/2021-22.csv')
teams = pd.read_csv('../../data/teams.csv')

# Splitting raw teams data into east and west conferences
conferences = { 'east' : list(teams[teams['location'] == 'east']['team']), 'west' : list(teams[teams['location'] == 'west' ]['team']) }

In [3]:
winning_teams = []
losing_teams = []

# Looping through all the games
for idx, game in data.iterrows():
    
    # Getting team names from data
    team1 = game['Visitor/Neutral']
    team2 = game['Home/Neutral']
    
    # Getting scores from data
    score1 = game['PTS']
    score2 = game['PTS.1']

    # If team1 won, add team1 to winning teams, and team 2 to losing teams
    if score1 > score2:
        winning_teams.append(team1)
        losing_teams.append(team2)
    
    # If team2 won, add team2 to winning teams, and team 1 to losing teams
    if score2 > score1:
        winning_teams.append(team2)
        losing_teams.append(team1)
        
# Initialising key-value data structure for amount of team wins
team_rankings = {"west":{},"east":{}}

# Counting instances of teams in winning and losing teams
wins = Counter(winning_teams)
losses = Counter(losing_teams)

# Populating data structure with win/loss values for Western conference teams
for team in conferences["west"]:
    team_rankings["west"][team] = {}
    team_rankings["west"][team]["Wins"] = wins[team]
    team_rankings["west"][team]["Losses"] = losses[team]
    team_rankings["west"][team]["W-L ratio"] = wins[team] / (wins[team]+losses[team])

# Populating data structure with win/loss values for Eastern conference teams
for team in conferences["east"]:
    team_rankings["east"][team] = {}
    team_rankings["east"][team]["Wins"] = wins[team]
    team_rankings["east"][team]["Losses"] = losses[team]
    team_rankings["east"][team]["W-L ratio"] = wins[team] / (wins[team]+losses[team])

# Creating matrices from data structure
team_rankings_west_df = pd.DataFrame.from_dict(team_rankings["west"], orient='index').sort_values("Wins", ascending=False)
team_rankings_east_df = pd.DataFrame.from_dict(team_rankings["east"], orient='index').sort_values("Wins", ascending=False)

In [6]:
# Outputting Western conferences team rankings
team_rankings_west_df

,Wins,Losses,W-L ratio
Phoenix Suns,31,9,0.775000
Golden State Warriors,30,10,0.750000
Memphis Grizzlies,29,14,0.674419
Utah Jazz,28,14,0.666667
Dallas Mavericks,22,19,0.536585
Los Angeles Clippers,21,21,0.500000
Los Angeles Lakers,21,21,0.500000
Denver Nuggets,20,19,0.512821
Minnesota Timberwolves,20,21,0.487805
Sacramento Kings,17,27,0.386364


In [7]:
# Outputting Eastern conferences team rankings
team_rankings_east_df

,Wins,Losses,W-L ratio
Chicago Bulls,27,12,0.692308
Brooklyn Nets,26,14,0.650000
Miami Heat,26,15,0.634146
Milwaukee Bucks,26,17,0.604651
Cleveland Cavaliers,24,18,0.571429
Charlotte Hornets,23,19,0.547619
Philadelphia 76ers,23,17,0.575000
Washington Wizards,22,20,0.523810
Boston Celtics,21,21,0.500000
New York Knicks,21,21,0.500000


In [8]:
# Finding advancing teams (more than or equal to 20 wins)
advancing_teams_east = list(team_rankings_east_df[team_rankings_east_df["Wins"] >= 20].index)
advancing_teams_west = list(team_rankings_west_df[team_rankings_west_df["Wins"] >= 20].index)

# Getting all players eligible for the MVP
mvp_candidates = players[(players["TEAM"].isin(advancing_teams_west)) | (players["TEAM"].isin(advancing_teams_east))]

#Initialising fantasy points array
fantasy_points = []

# Calculating fantasy points for each eligible player
for idx, player in mvp_candidates.iterrows():
    fantasy_points.append(3*player["3PM"]+2*(player["FGM"]-player["3PM"])+player["FTM"]+1.2*player["REB"]+1.5*player["AST"]+2*player["STL"]+2*player["BLK"]-player["TO"])

# Adding fantasy points row to MVP candidates matrix
mvp_candidates["Fantasy Points"] = fantasy_points

# Ordering MVP candidates by Fantasy points to determine recipient
mvp_candidates = mvp_candidates.sort_values(by="Fantasy Points", ascending=False)

# Outputting MVP candidates matrix
mvp_candidates

/tmp/ipykernel_10085/1433996545.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvp_candidates["Fantasy Points"] = fantasy_points


,PLAYER,TEAM,FGM,3PM,FTM,REB,AST,TO,STL,BLK,Fantasy Points
8,Nikola Jokic,Denver Nuggets,342,51,139,481,238,124,46,27,1830.2
5,Giannis Antetokounmpo,Milwaukee Bucks,339,38,250,389,204,114,36,49,1794.8
0,Kevin Durant,Brooklyn Nets,375,66,225,264,205,106,27,33,1679.3
15,Russell Westbrook,Los Angeles Lakers,297,42,149,344,342,185,47,12,1643.8
13,James Harden,Philadelphia 76ers,228,85,247,282,347,166,46,24,1620.9
...,...,...,...,...,...,...,...,...,...,...,...
561,CJ Miles,Boston Celtics,0,0,0,0,0,0,0,0,0.0
560,Arnoldas Kulboka,Charlotte Hornets,0,0,0,0,0,0,0,0,0.0
582,Sam Dekker,Toronto Raptors,0,0,0,0,0,0,0,0,0.0
585,Tyler Hall,New York Knicks,0,0,0,0,0,0,0,0,0.0


In [9]:
# Outputting MVP
print("Season MVP from wins >= 20 method is " + list(mvp_candidates.head(1)["PLAYER"])[0])

Season MVP from wins >= 20 method is Nikola Jokic
